In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import matplotlib.pyplot as plt
import numpy as np

import kipy
from kipy.board import BoardLayer as BL
import kipy.common_types as kct
import kipy.board_types as kbt
import kipy.geometry as kg
from kipy.proto.board import board_commands_pb2, board_types_pb2
from kipy.proto.common import base_types_pb2
import shapely
import shapely.geometry as sg
import shapely.plotting as sp

import breakneck.conversions
import breakneck.footprint
import breakneck.track


In [3]:
kicad = kipy.KiCad()


In [4]:
board = kicad.get_board()


In [5]:
fps = board.get_footprints()


ValueError: Can't unpack type.googleapis.com/kiapi.board.types.FootprintInstance.  Incompatible change on KiCad side?

In [317]:
vias = board.get_vias()


In [318]:
via0 = vias[0]
via0


Via(position=Vector2(53000000, 63000000), net=/Power/Supercaps/Supercap stage2/R+, type=VT_THROUGH, locked=False)

In [319]:
all_pads = board.get_pads()
gnd_tht_pads = [pad for pad in all_pads if pad.net.name == 'GND' and pad.proto.type == kbt.PadType.PT_PTH]


In [320]:
# We treat GND THT pads as vias as well
gnd_vias = [via for via in vias if via.net.name == 'GND'] + gnd_tht_pads


In [321]:
gnd_via_coords = [breakneck.conversions.vector2_as_coords(via.position) for via in gnd_vias]
gnd_via_points = [sg.Point(coords) for coords in gnd_via_coords]


In [322]:
gnd_via_tree = shapely.STRtree(gnd_via_points)


In [323]:
non_gnd_vias = [via for via in vias if via.net.name != 'GND']
non_gnd_via_coords = [breakneck.conversions.vector2_as_coords(via.position) for via in non_gnd_vias]
non_gnd_via_points = [sg.Point(coords) for coords in non_gnd_via_coords]


In [324]:
nearests_idxs = [gnd_via_tree.nearest(pt) for pt in non_gnd_via_points]


In [325]:
nearests = [gnd_via_points[idx] for idx in nearests_idxs]


In [326]:
shapely.distance(nearests[0], non_gnd_via_points[0])


np.float64(14159802.258506296)

In [327]:
distances = [shapely.distance(nearest, non_gnd_via) for nearest, non_gnd_via in zip(nearests, non_gnd_via_points)]


In [328]:
# Get those non-gnd vias that are more than 3000000 nm away from the nearest gnd via
lone_via_idxs = [i for i, distance in enumerate(distances) if distance > 2000000]


In [329]:
import kipy.board
import kipy.common_types
import kipy.proto


new_lines = []
for idx in lone_via_idxs:
    # Draw a line from the lone via to the nearest gnd via on layer BL.BL_Eco2_User
    # with thickness 0.05mm = 50000 nm
    ngvc = non_gnd_via_coords[idx]
    gvc = gnd_via_coords[nearests_idxs[idx]]
    start_proto_vector = base_types_pb2.Vector2(x_nm=ngvc[0], y_nm=-ngvc[1])
    end_proto_vector = base_types_pb2.Vector2(x_nm=gvc[0], y_nm=-gvc[1])

    shape = base_types_pb2.GraphicShape(
        segment=base_types_pb2.GraphicSegmentAttributes(
            start=start_proto_vector,
            end=end_proto_vector)
    )

    board_graphic_shape = board_types_pb2.BoardGraphicShape(shape=shape)

    segment = kbt.BoardSegment(proto=board_graphic_shape)
    segment.layer = BL.BL_Eco2_User
    segment.attributes.stroke.width = 100000
    new_lines.append(segment)


In [330]:
shapes = board.get_shapes()
eco2_shapes = [shape for shape in shapes if shape.layer == BL.BL_Eco2_User]


In [331]:
commit = board.begin_commit()
board.remove_items(eco2_shapes)
board.create_items(new_lines)
board.push_commit(commit)
